# CMDA 3654 Project - Money Ball 1

Marcos Fassio Bazzi - marcosfassiob18

Peter Do - kyungwan

Grant Collier - gcollier24

Ajay Kanjoor - ajkanjoor03

# Question: Which MLB teams will make the 2023 World Series?

Our goal for this project is to try to predict the 2023 MLB regular season and determine who will make the World Series based off of the data we can find [Baseball Reference](https://www.baseball-reference.com/) and [Spotrac](https://www.spotrac.com/mlb/payroll/2022/), two websites dedicated to baseball statistics and payroll, respectively.

## How does the MLB Regular Season work and how do teams make the World Series?

Major League Baseball is split up into two leagues: National League and American League. Each league is split up into 3 divisions: East, Central, and West. And each division has 5 teams in it making a total of 30 teams. Each team plays the four other teams in their division 19 times, and the 10 other teams in their league but not in their division either 6 or 7 times. This totals to a 162 game season for every team. The playoffs have changed throughout the years, but this year a total of 12 teams make the playoffs, 6 from both leagues. To win your division you need to have the best record out of the five teams, and the winners of each of the 6 division make the playoffs. In addition, out of all the teams that didn't win their division, the 3 teams with the best records in both leagues make the playoffs. In the playoffs, the two teams that win their league make it to the World Series. The 1 and 2 seed from both leagues get a by and automatically make it to the quarter finals, so those teams have an advantage.

# Scraping, cleaning, grouping and visualizing the data

Exporting the data from Baseball Reference is easy enough - the site has an option that allows us to share and export as a `.csv` file. However, the same cannot be said for Spotrac. In this case, we used Python libraries `requests` and `BeautifulSoup4` to fetch and scrape the payroll data from Spotrac.

## Importing necessary libraries

In [43]:
import requests
import bs4
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

## Scraping the 2022 payroll data

In [255]:
# fetch link and get page
url = "https://www.spotrac.com/mlb/payroll/2022/"
header = { "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" }
link = requests.get(url, headers=header)
soup = bs4.BeautifulSoup(link.content, "html5lib")

## get table and parse headers
table_data = soup.find('table', { "class": "datatable" })
headers = [i.text.strip() for i in table_data.find_all("th")]

## parse table body
table_body = soup.find("tbody")
team_data = []
for row_data in table_body.find_all("tr"):
    team_data += [[entry.text.strip().replace("\n", "").replace("\t", "") for entry in row_data.find_all("td")]]

# clean team names
for i in range(len(team_data)):
    team_data[i][1] = team_data[i][1][:-3] # last three characters are the team's abbreviation - we don't want this
team_data[4][1] = "San Diego Padres" # for whatever reason this entry's abbreviation was two characters instead of three
team_data.pop(14)
team_data.pop(14) # get rid of league averages in table

# turn data into dataframe
payroll_data_2022 = pd.DataFrame(team_data, columns=headers)
payroll_data_2022.head()

,Rank,Team,Win%,Roster,26-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2022 Total Payroll
0,1,Los Angeles Dodgers,0.685,28,"$231,003,922","$28,920,266","$5,332,866","$7,040,000","$35,333,333","$270,381,426"
1,2,New York Mets,0.623,31,"$231,062,889","$4,155,954","$28,130,999","$14,468,000",0-,"$268,292,506"
2,3,New York Yankees,0.611,30,"$214,676,803","$25,203,665","$12,115,094","$9,307,500",0-,"$252,957,200"
3,4,Philadelphia Phillies,0.537,28,"$191,757,795","$11,897,332","$30,945,214","$17,478,000",0-,"$244,484,097"
4,5,San Diego Padres,0.549,29,"$173,934,032","$10,925,000","$34,528,348","$6,304,700","$5,714,286","$224,511,694"


## Scraping the 2023 payroll data

The process is identical.

In [256]:
# fetch link and get page
url = "https://www.spotrac.com/mlb/payroll/2023/"
header = { "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" }
link = requests.get(url, headers=header)
soup = bs4.BeautifulSoup(link.content, "html5lib")

## get table and parse headers
table_data = soup.find('table', { "class": "datatable" })
headers = [i.text.strip() for i in table_data.find_all("th")]

## parse table body
table_body = soup.find("tbody")
team_data = []
for row_data in table_body.find_all("tr"):
    team_data += [[entry.text.strip().replace("\n", "").replace("\t", "") for entry in row_data.find_all("td")]]

# clean team names
for i in range(len(team_data)):
    team_data[i][1] = team_data[i][1][:-3] # last three characters are the team's abbreviation - we don't want this
team_data[4][1] = "San Diego Padres" # for whatever reason this entry's abbreviation was two characters instead of three
team_data.pop(14)
team_data.pop(14) # get rid of league averages in table

# turn data into dataframe
payroll_data_2023 = pd.DataFrame(team_data, columns=headers)
payroll_data_2023.head()

,Rank,Team,Win%,Roster,26-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2023 Total Payroll
0,1,New York Mets,0.500,26,"$227,124,999","$74,608,333","$34,500,000",0-,0-,"$336,233,332"
1,2,New York Yankees,0.600,27,"$208,570,714","$60,383,333",0-,0-,0-,"$268,954,047"
2,3,San Diego Padre,0.500,26,"$176,820,454","$41,530,000","$12,280,000",0-,"$7,714,286","$236,962,024"
3,4,Philadelphia Phillies,0.200,26,"$181,686,477","$42,488,462",0-,"$8,250,000",0-,"$232,424,939"
4,5,San Diego Padres,0.667,26,"$167,720,000","$28,075,000","$22,537,634",0-,0-,"$218,332,634"


## Fetching the 2022 MLB team data from Baseball Reference

First, we copied the data as a `.csv` file and exported it.

In [262]:
baseball_data_2022 = pd.read_csv("baseball_data_2022.csv")
baseball_data_2022.head()

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,Arizona Diamondbacks,57,26.5,4.33,162,6027,5351,702,1232,262,...,0.385,0.689,95,2061,97,60,31,50,14,1039
1,Atlanta Braves,53,27.5,4.87,162,6082,5509,789,1394,298,...,0.443,0.761,111,2443,103,66,1,36,13,1030
2,Baltimore Orioles,58,27.0,4.16,162,6049,5429,674,1281,275,...,0.390,0.695,97,2119,95,83,12,43,10,1095
3,Boston Red Sox,54,28.8,4.54,162,6144,5539,735,1427,352,...,0.409,0.731,102,2268,131,63,12,50,23,1133
4,Chicago Cubs,64,27.9,4.06,162,6072,5425,657,1293,265,...,0.387,0.698,96,2097,130,84,19,36,16,1100
